# Setup

In [1]:
import os
import pandas as pd
import json
import tqdm

In [2]:
# data paths:
DATA_ROOT = "/proj/systewar/datasets/audioset_music_mood/audio_files"
ONTOLOGY_FILE = "/proj/systewar/datasets/audioset_music_mood/orig_metadata_files/ontology.json"
METADATA_FILES = {
    "train": "/proj/systewar/datasets/audioset_music_mood/orig_metadata_files/unbalanced_train_segments.csv",
    "eval": "/proj/systewar/datasets/audioset_music_mood/orig_metadata_files/eval_segments.csv"
}
# AudioSet music mood subset label names:
MOOD_LABEL_NAMES = ["Happy music", "Funny music", "Sad music", "Tender music", "Exciting music", "Angry music", "Scary music"]

In [9]:
# script options:
# data_subsets = ["train", "eval"]
data_subsets = ["eval"]
label_files = {
    "train": "/proj/systewar/datasets/audioset_music_mood/labels_train.csv",
    "eval": "/proj/systewar/datasets/audioset_music_mood/labels_eval.csv"
}

# Get Label IDs

In [4]:
# load ontology dictionary:
with open(ONTOLOGY_FILE, "r") as json_file:
    ontology = json.load(json_file)

In [5]:
# get label IDs of music mood labels:
mood_label_ids = {}
for entry in ontology:
    if entry["name"] in MOOD_LABEL_NAMES:
        if entry["name"] in mood_label_ids.values():
            raise RuntimeError("'{}' already found in ontology.".format(entry["name"]))
        else:
            mood_label_ids[entry["id"]] = entry["name"]

print("Music mood label ids and names:\n")
for label_name, label_id in mood_label_ids.items():
    print("{}: {}".format(label_name, label_id))

Music mood label ids and names:

/t/dd00031: Happy music
/t/dd00032: Funny music
/t/dd00033: Sad music
/t/dd00034: Tender music
/t/dd00035: Exciting music
/t/dd00036: Angry music
/t/dd00037: Scary music


# Extract Labels

In [6]:
# load metadata:
metadata_dfs = {}
col_names = ["youtube_id", "start_seconds", "end_seconds", "positive_labels"]
for subset in data_subsets:
    print("Loading {} set metadata...\n".format(subset))
    metadata_dfs[subset] = pd.read_csv(METADATA_FILES[subset], sep=", ", names=col_names, header=None, skiprows=[0, 1, 2])
    print(metadata_dfs[subset].info())
    print("\n")

Loading eval set metadata...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20371 entries, 0 to 20370
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   youtube_id       20371 non-null  object 
 1   start_seconds    20371 non-null  float64
 2   end_seconds      20371 non-null  float64
 3   positive_labels  20371 non-null  object 
dtypes: float64(2), object(2)
memory usage: 636.7+ KB
None




/tmp/ipykernel_19969/2926517863.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  metadata_dfs[subset] = pd.read_csv(METADATA_FILES[subset], sep=", ", names=col_names, header=None, skiprows=[0, 1, 2])


In [7]:
# extract audio file names:
audio_file_names = {}
for subset in data_subsets:
    subset_root = os.path.join(DATA_ROOT, subset)
    audio_file_names[subset] = [name for name in os.listdir(subset_root) if os.path.isfile(os.path.join(subset_root, name))]
    print("Original size of {} set: {}".format(subset, len(audio_file_names[subset])))

Original size of eval set: 344


In [10]:
# extract labels of audio files:
label_dfs = {}
for subset in data_subsets:
    file_names = []
    mood_labels = []
    n_bad_files = 0     # number of audio files with not exactly 1 music mood label
    for file_name in tqdm.tqdm(audio_file_names[subset], total=len(audio_file_names[subset]), desc="Extracting labels for {} set".format(subset)):
        # extract youtube ID from file name:
        youtube_id_start_time = file_name.replace(".wav", "")
        assert len(youtube_id_start_time) == len(file_name) - len(".wav"), "Error with removing '.wav'."
        start_time = "_" + youtube_id_start_time.split("_")[-1]
        youtube_id = youtube_id_start_time.replace(start_time, "")
        assert len(youtube_id) == len(youtube_id_start_time) - len(start_time), "Error with removing start_time."

        # get label ids by querying metadata with youtube id:
        label_ids = metadata_dfs[subset]["positive_labels"][metadata_dfs[subset]["youtube_id"] == youtube_id]

        # convert format of labels ids:
        label_ids = label_ids.reset_index(drop=True)
        label_ids = label_ids[0]
        # remove leading/trailing quotation marks:
        orig_len = len(label_ids)
        label_ids = label_ids.strip('"')
        assert len(label_ids) == orig_len - 2, "Error with removing leading/trailing quotation marks from labels string."
        # convert to list:
        label_ids = label_ids.split(",")
        assert type(label_ids) == list, "Error with converting labels string to list."

        # extract music mood label (should be exactly 1):
        mood_label = None
        n_mood_labels = 0
        for label_id in label_ids:
            if label_id in mood_label_ids.keys():
                mood_label = mood_label_ids[label_id]
                n_mood_labels += 1
        # only keep audio files with exactly 1 music mood label:
        if n_mood_labels == 1:
            file_names.append(subset + "/" + file_name)
            mood_labels.append(mood_label)
        else:
            n_bad_files += 1
    
    print("Number of audio files with not exactly 1 music mood label: {}".format(n_bad_files))
    
    # save as dataframe:
    label_dfs[subset] = pd.DataFrame(
        data={
            "file_name": file_names,
            "label": mood_labels
        }
    )
    print()
    print(label_dfs[subset].info())
    print()
    print(label_dfs[subset].head())
    print("\n")
    # save to file:
    label_dfs[subset].to_csv(label_files[subset], index=False)

Extracting labels for eval set: 100%|██████████| 344/344 [00:00<00:00, 419.10it/s]

Number of audio files with not exactly 1 music mood label: 7

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  337 non-null    object
 1   label      337 non-null    object
dtypes: object(2)
memory usage: 5.4+ KB
None

                    file_name           label
0   eval/rez5KDmIZoc_30.0.wav    Tender music
1   eval/ogW2tWCxpPM_30.0.wav     Funny music
2    eval/1bSP4wLfMpA_0.0.wav     Scary music
3   eval/8akqVDDGsNQ_30.0.wav  Exciting music
4  eval/sC7T0sEG6ek_230.0.wav     Angry music


